---

# Supplementary notebook for leaf wood segmentation paper


August 2023 \
Wouter Van den Broeck

---

## 1. Evaluating leaf wood segmentation results derived from the LeWoS method

In [1]:
# Load packages
import numpy as np
import os

In [52]:
# Specify data paths
data_path =  '/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/leaf_wood/'
dir_true = 'preprocessed/'
dir_pred = 'preprocessed_open3d/test_pred_LeWOS/'
path_true = os.path.join(data_path, dir_true)
path_pred = os.path.join(data_path, dir_pred)

filenames = os.listdir(path_pred)
print('number of prediction files:', len(filenames))

# Loop over ground truth and predictions and keep track of metrics
TP, FP, TN, FN = 0, 0, 0, 0

for filename in filenames:
    # Load ground truth and prediction
    gt = np.loadtxt(os.path.join(path_true, filename))
    try:
        pred = np.loadtxt(os.path.join(path_pred, filename))
    except:
        raise Warning(f'No ground truth file found with name {filename}')

    if len(gt) != len(pred):
        raise Exception(f'Length of prediction file and ground truth file ({filename}) do not match!')

    gt = gt[gt[:, 0].argsort()]
    gt = gt[gt[:, 1].argsort(kind='mergesort')]
    gt = gt[gt[:, 2].argsort(kind='mergesort')]
    pred = pred[pred[:, 0].argsort()]
    pred = pred[pred[:, 1].argsort(kind='mergesort')]
    pred = pred[pred[:, 2].argsort(kind='mergesort')]

    # get label
    gt = gt[:, 3].flatten()
    pred = pred[:, 3].flatten()

    TP += ((gt == 1) & (pred == 1)).sum()
    TN += ((gt == 0) & (pred == 0)).sum()
    FP += ((gt == 0) & (pred == 1)).sum()
    FN += ((gt == 1) & (pred == 0)).sum()

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
iou = TP / (TP + FN + FP)
f1 = (2 * TP) / (2 * TP + FP + FN)

print(
    'TP:', TP, 'TN:', TN, 'FP:', FP, 'FN:', FN, '\n'
    'accuracy:', accuracy, '\n',
    'precision:', precision, '\n',
    'recall:', recall, '\n',
    'iou:', iou, '\n',
    'f1:', f1, '\n',
)

number of prediction files: 30
TP: 1264357 TN: 5451431 FP: 87359 FN: 367135 
accuracy: 0.9366142084788297 
 precision: 0.9353717792790793 
 recall: 0.7749697822606547 
 iou: 0.7355826653968262 
 f1: 0.8476492420240225 



## Code for converting txt files to ply files

In [19]:
import open3d as o3d

data_path =  '/mnt/c/Users/wavdnbro/OneDrive - UGent/Documents/spacetwin/datasets/leaf_wood/'
filenames = [f[:-3] + 'txt' for f in os.listdir(data_path + 'preprocessed_open3d/test') if f[-3:] == 'npy']

for filename in filenames:
    # Read file
    pcl = np.loadtxt(os.path.join(data_path, 'preprocessed', filename))
    xyz = pcl[:, :3]
    labels = pcl[:, -1].reshape(-1, 1)

    # Convert to open3d tensor format
    device = o3d.core.Device("CPU:0")
    pcd = o3d.t.geometry.PointCloud(device)
    pcd.point.positions = o3d.core.Tensor(xyz, o3d.core.float32, device)
    pcd.point.labels = o3d.core.Tensor(labels, o3d.core.int32, device)

    # Write as ply file
    file_out = os.path.join(data_path, 'preprocessed_open3d/test_ply', filename[:-3] + 'ply')
    o3d.t.io.write_point_cloud(file_out, pcd, write_ascii=False, compressed=False, print_progress=False)


    


